## Import Libraries
In order to generate a bag of words representation we need the following libraries

In [ ]:
import numpy as np
import json
import pandas as pd

## Import Data


In [ ]:
# data_set = np.array([ [x['venue'], x['citations']] for x in json.load(open('train-1.json'))[:3] if x['venue']])
data_set = np.array([ [x['venue'], x['citations']] for x in json.load(open('train-1.json')) if x['venue']])
# print(data_set[0])

## Proportion analysis of journals/venues
Aim of the code is to divide the data in n equal portions, where the 'lowest' division accounts for the group of X that contributes the least to a given Y.

In [73]:
def get_cat(dataFrame):
    conditions = [
        (dataFrame['cumsum'] == 0),
        (dataFrame['cumsum'] > 0) & (dataFrame['cumsum'] <= 25),
        (dataFrame['cumsum'] > 25) & (dataFrame['cumsum'] <= 50),
        (dataFrame['cumsum'] > 50) & (dataFrame['cumsum'] <= 75),
        (dataFrame['cumsum'] >= 75)
    ]
    values = [1, 2, 3, 4,5]
    dataFrame['class_Venue'] = np.select(conditions, values)
    return dataFrame

def get_venue_dictionary(XY):
    #split data 
    X, _ = np.split(XY, 2, 1)
    
    unique_X = np.unique(X,return_counts=True)
    unique_X_length = len(unique_X[0])
    print(unique_X_length)
    zero_array = np.zeros((3,unique_X_length))

    merge = [unique_X[0], unique_X[1].astype(int), zero_array[0].astype(int), zero_array[1].astype(int), zero_array[2].astype(float)]
    
    # Compute citations per topic
    for x in XY :
        merge[2][np.where(merge[0] == x[0])[0][0]] += int(x[1])

    # Compute citations per topic divided by the amount of articles
    for (i, j) in enumerate(merge[3]):
        merge[3][i] = (merge[2][i]/merge[1][i])

    # Compute new summed citations
    summed_citations = np.sum(merge[3])
    result = []
    for (i, j) in enumerate(merge[3]):
        merge[4][i] = 100/summed_citations*merge[3][i]
        result.append((merge[0][i],merge[1][i],merge[2][i],merge[3][i],merge[4][i]))
    
    dtype = [('field_of_study', 'S100'), ('count', int), ('summed_citations', int), ('average_citations', float), ('contribution', float)]
    structured_array = np.array(division, dtype=dtype)
    sorted = np.sort(structured_array, order='contribution')
    df = pd.DataFrame(sorted)
    df['cumsum'] = df['contribution'].cumsum(axis=0)
    df = get_cat(df)
    # return {x['field_of_study'] : x['class_Venue'] for x in df}
    return df
    # return (df['field_of_study'],df['class_Venue'])
    
get_venue_dictionary(data_set).to_pickle("venue_data.pkl")

299


In [ ]:
def divide_data_in_portions(XY):
    #split data
    XY_split = np.split(XY, 2, 1)
    Y = XY_split[1].astype(np.int16)
    
    summed_Y = np.sum(Y)
    print('There are in total', summed_Y, 'citations in this data')    
    unique_X = np.unique(XY_split[0],return_counts=True)
    print('Found',len(unique_X[0]), 'unique variables')
    a = np.zeros((3,len(unique_X[0])))
    c = [unique_X[0], unique_X[1].astype(int), a[0].astype(int), a[1].astype(int), a[2].astype(float)]
    for x in XY :
        c[2][np.where(c[0] == x[0])[0][0]] += int(x[1])
    for (i, j) in enumerate(c[3]):
        c[3][i] = (c[2][i]/c[1][i])
        c[4][i] = 100/summed_Y*c[2][i]
    return c
    

division = divide_data_in_portions(data_set)
sorted_division = np.sort(division[3])

print (division)

## Plot the line
In order to make the data more acessible, the data can be plotted

In [61]:
division = divide_data_in_portions(data_set)
dtype = [('field_of_study', 'S100'), ('count', int), ('summed_citations', int), ('average_citations', float), ('contribution', float)]

# create a structured array
structured_array = np.array(division, dtype=dtype)
sorted = np.sort(structured_array, order='contribution')

# plt.scatter(range(len(sorted_division)), sorted_division)

import pandas as pd
import numpy as np




print(df)


299
     field_of_study  count  summed_citations  average_citations  contribution  \
0       b'ACL 1979'      1                 0                0.0      0.000000   
1           b'ALVR'      7                 6                0.0      0.000000   
2    b'AMERICASNLP'     15                 6                0.0      0.000000   
3           b'BPPF'      3                 0                0.0      0.000000   
4        b'CLPSYCH'      8                 2                0.0      0.000000   
..              ...    ...               ...                ...           ...   
294      b'ALW@ACL'      7               803              114.0      2.038262   
295   b'EMNLP 2019'      1               144              144.0      2.574647   
296        b'IJCAI'      1               169              169.0      3.021634   
297   b'EMNLP 2017'      2               387              193.0      3.450742   
298   b'SSST@EMNLP'      8              3659              457.0      8.170928   

         cumsum  class_